In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *


from PIL import Image
from numpy import array
import cv2

In [3]:
from math import atan2
import torch

In [4]:
fileNum = 1
seg_layer = 0
X = 0
Y = 1

vector_num = 0
vectors_array = []
one_image = []

# Vector Conversion: cartesian-to-polar
- Insert vectors into a list

In [5]:
# vectors[vector index][segmentation layer][x or y coordinate]


# This loop controls the file number 1-21
while fileNum < 22:
    
    vectors = np.load(f'vectorList{fileNum}.npy')
    vect_length = len(vectors)
    vect_index = 0
    
    # This loop controls which vector is being converted, ~998-1000 vectors per file
    while vect_index < vect_length:
        
        seg_layer = 0
        one_image = torch.ones(32, 2)
        
        # This loop iterates through all segmentation layers for ONE image
        while seg_layer < 32:
            
            # Convert to Magnitude
            mag = ((vectors[vect_index][seg_layer][X] *vectors[vect_index][seg_layer][X]) + (vectors[vect_index][seg_layer][Y] * vectors[vect_index][seg_layer][Y]))
            mag = math.sqrt(mag)
            mag = math.radians(mag)
            
            # Convert to Angle
            ang = atan2(vectors[vect_index][seg_layer][Y], vectors[vect_index][seg_layer][X])
            ang = math.radians(ang)
            
            # Add to one_image list
            one_image[seg_layer][X] = mag
            one_image[seg_layer][Y] = ang
            
            seg_layer += 1
        
        
        vectors_array.insert(0, one_image)
        vect_index += 1

    print('file no. : ', fileNum)
    fileNum += 1

file no. :  1
file no. :  2
file no. :  3
file no. :  4
file no. :  5
file no. :  6
file no. :  7
file no. :  8
file no. :  9
file no. :  10
file no. :  11
file no. :  12
file no. :  13
file no. :  14
file no. :  15
file no. :  16
file no. :  17
file no. :  18
file no. :  19
file no. :  20
file no. :  21


In [6]:
vectors_array.reverse()
len(vectors_array)

20399

In [7]:
vector_tensor = torch.ones(20399, 32, 2)
len(vector_tensor)

20399

In [8]:
vectors_array[0]

tensor([[ 0.0000e+00,  0.0000e+00],
        [ 2.4149e-07,  3.9406e-03],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 7.0389e-03, -2.0664e-02],
        [ 4.8853e-05,  6.6832e-03],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 2.3607e-04,  2.0237e-03],
        [ 7.1059e-05,  1.8003e-02],
        [ 0.0000e+00,  0.0000e+00],
        [ 5.4655e-04, -2.8307e-02],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 2.0026e-05,  5.2533e-02],
        [ 7.4990e-07,  8.9811e-04],
        [ 1.2084e-03,  2.0808e-02],
        [ 0.0000e+00,  0.0000e+00],
        [ 3.4511e-04,  5.0593e-02],
        [ 0.0000e+00,  0.0000e+00],
        [ 7.0257e-08, -3.1267e-02],
        [ 1.9098e-05,  1.3820e-02],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 4.9730e-06,  3.6198e-03],
        [ 2.5107e-04, -2.146

### Transfer all the info to a tensor 

In [9]:
for i,j in enumerate(vectors_array):
    vector_tensor[i] = vectors_array[i]

In [10]:
vector_tensor[0]

tensor([[ 0.0000e+00,  0.0000e+00],
        [ 2.4149e-07,  3.9406e-03],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 7.0389e-03, -2.0664e-02],
        [ 4.8853e-05,  6.6832e-03],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 2.3607e-04,  2.0237e-03],
        [ 7.1059e-05,  1.8003e-02],
        [ 0.0000e+00,  0.0000e+00],
        [ 5.4655e-04, -2.8307e-02],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 2.0026e-05,  5.2533e-02],
        [ 7.4990e-07,  8.9811e-04],
        [ 1.2084e-03,  2.0808e-02],
        [ 0.0000e+00,  0.0000e+00],
        [ 3.4511e-04,  5.0593e-02],
        [ 0.0000e+00,  0.0000e+00],
        [ 7.0257e-08, -3.1267e-02],
        [ 1.9098e-05,  1.3820e-02],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 4.9730e-06,  3.6198e-03],
        [ 2.5107e-04, -2.146

In [11]:
len(vector_tensor)

20399

# Load Labels (speeds)

In [12]:
with open('train.txt', mode='r') as f:
    x = f.readlines()

In [13]:
lab = [] 

for i in range(len(x)-1):
    lab.insert(0, float(x[i]))

lab.reverse()
len(lab)

20399

In [14]:
type(lab[1])

float

In [157]:
# labels = np.loadtxt('train.txt', dtype=float)
# len(labels)

20400

# Split up data (training set, valid set, test set)

In [16]:
TRAIN_PERCENT = .7
VALID_PERCENT = .2
NUM_DATA_INSTANCES = len(lab)

NUM_TRAIN_INSTANCES = int(TRAIN_PERCENT * NUM_DATA_INSTANCES)
NUM_VALID_INSTANCES = int(VALID_PERCENT * NUM_DATA_INSTANCES)
NUM_TEST_INSTANCES = int(NUM_DATA_INSTANCES - NUM_TRAIN_INSTANCES - NUM_VALID_INSTANCES)

In [17]:
NUM_VALID_INSTANCES

4079

In [ ]:
train_idx = vectors_array[:NUM_TRAIN_INSTANCES]

In [20]:
print(len(train_idx))

14279


In [21]:
valid_idx = vectors_array[NUM_TRAIN_INSTANCES: (NUM_DATA_INSTANCES - NUM_TEST_INSTANCES)]
len(valid_idx)

4079

In [22]:
test_idx = vectors_array[(NUM_DATA_INSTANCES - NUM_TEST_INSTANCES):]
len(test_idx)

2041

In [23]:
train_idx[14000].shape

torch.Size([32, 2])

In [24]:
FEATURE_DIM = train_idx[0].size()

In [25]:
FEATURE_DIM

torch.Size([32, 2])

## Alt way of setting Feature_dim var

In [26]:
# CREATE A tupe of INTS
# FEATURE_DIM = (FEATURE_DIM[0], FEATURE_DIM[1])

In [27]:
FEATURE_DIM

torch.Size([32, 2])

In [28]:
type(FEATURE_DIM[1])

int

# Prepare labels for each set

In [29]:
trainLabels = lab[:NUM_TRAIN_INSTANCES]
validLabels = lab[NUM_TRAIN_INSTANCES : (NUM_DATA_INSTANCES - NUM_TEST_INSTANCES)]
testLabels = lab[(NUM_DATA_INSTANCES - NUM_TEST_INSTANCES):]

In [30]:
type(trainLabels)

list

In [31]:
lab[0]

28.105569

# Pair training label with training data

In [32]:
train_data = [] 
for i in range(len(train_idx)):
    train_data.append([train_idx[i], lab[i]])

In [33]:
len(train_data)

14279

### Create a training set dataloader

In [34]:
trainLoader = torch.utils.data.DataLoader(train_data)

### Repeat process with valid-set, test-set

In [35]:
len(valid_idx)

4079

In [36]:
len(validLabels)

4079

In [37]:
valid_data = [] 
for i in range(len(valid_idx)):
    valid_data.append([valid_idx[i], validLabels[i]])

In [38]:
validLoader = torch.utils.data.DataLoader(valid_data)

In [40]:
len(testLabels)

2041

In [41]:
test_data = [] 
for i in range(len(test_idx)):
    valid_data.append([valid_idx[i], testLabels[i]])

In [42]:
testLoader = torch.utils.data.DataLoader(test_data)

# Lstm Class

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [44]:
class LSTM(nn.Module):
    
    # input = 32 bc there's 32 features
    # hidden_layer = ??
    # output size = 1, bc we want to predict the speed for one frame
    
    
    def __init__(self, input_size=64, hidden_layer_size=100, output_size=32):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [45]:
FEATURE_DIM

torch.Size([32, 2])

In [64]:
FEATURE_DIM = 64

FEATURE_DIM

64

In [65]:
model = LSTM(input_size=FEATURE_DIM, hidden_layer_size=100, output_size=32)

### Set loss function, set learning rate

In [66]:
LEARNING_RATE = 0.001

In [67]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= LEARNING_RATE)

## Question: 

According to the pytorch docs:
- loss_function = nn.MSELoss(input, output)


input and output need to be the same type and format

Right now:
- Input = y_pred = tensor[float] 32 x 2
- Output = labels = 1 x 1 float


Documentation: 
- https://pytorch.org/docs/stable/nn.html#mseloss

### Set number of epochs, train the model

In [68]:
epochs = 10
# epochs = 150

for i in range(epochs):
    
    for seq, labels in trainLoader:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size), 
                            torch.zeros(1, 1, model.hidden_layer_size))
        print('seq: ', seq)
        y_pred = model(seq)
        print('y_pred: ', y_pred)
        
        print('len: y_pred ', len(y_pred))
        print('labels: ', labels)
        
        
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
    
    if i % 25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item(): 10.8f}')
        
print(f'epoch: {i:3} loss: {single_loss.item(): 10.10f}')

seq:  tensor([[[ 0.0000e+00,  0.0000e+00],
         [ 2.4149e-07,  3.9406e-03],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 7.0389e-03, -2.0664e-02],
         [ 4.8853e-05,  6.6832e-03],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 2.3607e-04,  2.0237e-03],
         [ 7.1059e-05,  1.8003e-02],
         [ 0.0000e+00,  0.0000e+00],
         [ 5.4655e-04, -2.8307e-02],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 2.0026e-05,  5.2533e-02],
         [ 7.4990e-07,  8.9811e-04],
         [ 1.2084e-03,  2.0808e-02],
         [ 0.0000e+00,  0.0000e+00],
         [ 3.4511e-04,  5.0593e-02],
         [ 0.0000e+00,  0.0000e+00],
         [ 7.0257e-08, -3.1267e-02],
         [ 1.9098e-05,  1.3820e-02],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 4.9730e-06,  3.6198e-

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #2 'target'